In [4]:
## convert to function
from imutils import contours
from skimage import measure
import numpy as np
import imutils
import cv2
import pandas as pd
import glob
from tqdm.auto import tqdm
tqdm.pandas()

import pandas as pd
import shutil

In [5]:

def get_disk(image):
  try:  
    image = cv2.imread(image)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.bilateralFilter(gray,9,75,75)
    median=cv2.medianBlur(blur, 5)

    # threshold the image to reveal light regions in the
    # blurred image
    thresh = cv2.threshold(median, 155, 255, cv2.THRESH_BINARY)[1]
    # perform a series of erosions and dilations to remove
    # any small blobs of noise from the thresholded image
    thresh = cv2.erode(thresh, None, iterations=2)
    thresh = cv2.dilate(thresh, None, iterations=4)

    # perform a connected component analysis on the thresholded
    # image, then initialize a mask to store only the "large"
    # components
    labels = measure.label(thresh, connectivity=2, background=0)
    mask = np.zeros(thresh.shape, dtype="uint8")

    # loop over the unique components
    for label in np.unique(labels):
      # if this is the background label, ignore it
      if label == 0:
        continue

      # otherwise, construct the label mask and count the
      # number of pixels 
      labelMask = np.zeros(thresh.shape, dtype="uint8")
      labelMask[labels == label] = 255
      numPixels = cv2.countNonZero(labelMask)

      # if the number of pixels in the component is sufficiently
      # large, then add it to our mask of "large blobs"
      if numPixels >500:
        mask = cv2.add(mask, labelMask)

    # find the contours in the mask, then sort them from left to
    # right
    cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL,
      cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    cnts = contours.sort_contours(cnts)[0]
    # print (cnts)
    # loop over the contours
    for (i, c) in enumerate(cnts):
        ellipse = cv2.fitEllipse(c)
        (x, y, w, h) = cv2.boundingRect(c)
        break
      # draw the bright spot on the image

    padding = 100
    image = image[y-padding:y+h+padding, x-padding:x+w+padding]
    return True
  except:
    return False

In [6]:
df = pd.DataFrame(columns=["path", "raises_exception"])

In [7]:
path_list = (glob.glob("../dataset_generation/kaggle_dataset/0/*.*")) + (glob.glob("../dataset_generation/kaggle_dataset/1/*.*"))

In [8]:
df['path'] = path_list

In [9]:
df

,path,raises_exception
0,../dataset_generation/kaggle_dataset/0/189.jpg,NaN
1,../dataset_generation/kaggle_dataset/0/162.jpg,NaN
2,../dataset_generation/kaggle_dataset/0/176.jpg,NaN
3,../dataset_generation/kaggle_dataset/0/610.jpg,NaN
4,../dataset_generation/kaggle_dataset/0/360.jpg,NaN
...,...,...
645,../dataset_generation/kaggle_dataset/1/580.jpg,NaN
646,../dataset_generation/kaggle_dataset/1/345.jpg,NaN
647,../dataset_generation/kaggle_dataset/1/386.jpg,NaN
648,../dataset_generation/kaggle_dataset/1/609.jpg,NaN


In [10]:
df['raises_exception'] = df.path.progress_apply(get_disk)

In [14]:
df.raises_exception.value_counts()

True    650
Name: raises_exception, dtype: int64

In [15]:
df.to_csv('exceptions_kaggle.csv', index=False)